## Importing Dependencies

In [1]:
import tensorflow as tf
# from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,LocallyConnected2D
from keras.layers import Dense,Flatten, Dropout
# from keras.utils import to_categorical
# from keras import backend as K
import keras

## Importing Dependencies

In [2]:
from pathlib import Path
import numpy as np
import os
import cv2
import dlib

## Building model by using keras

In [3]:
model = Sequential()

In [4]:
model.add(Conv2D(32, (11, 11), activation='relu', name='C1', input_shape=(152, 152, 3)))

In [5]:
model.add(MaxPooling2D(pool_size=3, strides=2, padding='same', name='M2'))

In [6]:
model.add(Conv2D(16, (9, 9), activation='relu', name='C3'))

In [7]:
model.add(LocallyConnected2D(16, (9, 9), activation='relu', name='L4'))

In [8]:
model.add(LocallyConnected2D(16, (7, 7), strides=2, activation='relu', name='L5') )

In [9]:
model.add(LocallyConnected2D(16, (5, 5), activation='relu', name='L6'))

In [10]:
model.add(Flatten(name='F0'))

In [11]:
model.add(Dense(4096, activation='relu', name='F7'))

In [12]:
model.add(Dropout(rate=0.5, name='D0'))

In [13]:
model.add(Dense(8631, activation='softmax', name='F8'))

In [14]:
#Dropping last layer since we dont need it for our model
deepface_model = keras.models.Model(inputs=model.layers[0].input, outputs=model.layers[-3].output)

In [15]:
#Loading weight of pretrained model
model.load_weights("VGGFace2_weights.h5")

## Function for finding Euclidean Distance

In [16]:
def normalize(x):
    return x / np.sqrt(np.sum(np.multiply(x, x)))
def findEuclideanDistance(source_representation, test_representation):
    euclidean_distance = source_representation - test_representation
    euclidean_distance = np.sum(np.multiply(euclidean_distance, euclidean_distance))
    euclidean_distance = np.sqrt(euclidean_distance)
    return euclidean_distance

## Pre Processing

In [17]:
#converting image to array
def process_img(image_path):
    test_image=tf.keras.preprocessing.image.load_img(image_path,target_size=(152,152))
    test_image=tf.keras.preprocessing.image.img_to_array(test_image)
    test_image=np.expand_dims(test_image,axis=0)
    return test_image

## Making Embedding Matrix

In [18]:
#Run only if there is a change in dataset
Embedding_List = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using deepface
Name_List = []  # list of names corrospoing to cropped photos
for folderPath in os.listdir('dataset'): 
    fpath=os.path.join('dataset',folderPath) #traversing through all folders in the dataset
    for imgpath in os.listdir(fpath): #traversing through all images in the sub folders of dataset
        imagepath=os.path.join(fpath,imgpath)
        Embedding_List.append(deepface_model.predict(process_img(imagepath))[0])
        Name_List.append(folderPath)
        
#saving data
data = [Name_List, Embedding_List]
np.save('data.npy', data)

1/1 [==============================] - 0s 43ms/step


In [21]:
#Loading data
load_data=np.load('data.npy', allow_pickle=True)
name_list=load_data[0]
embedding_list = load_data[1]

In [22]:
for i in range (len(name_list)): #printing name and corresponding embedding matrix
    print(name_list[i]) 
    print(embedding_list[i])

Avishkar Pawar
[  0.        0.      367.02005 ...   0.        0.        0.     ]
Avishkar Pawar
[  0.      275.09354   0.      ...   0.        0.        0.     ]
Avishkar Pawar
[0. 0. 0. ... 0. 0. 0.]
Avishkar Pawar
[  0.        0.        0.      ...   0.      396.6325  330.40582]
Avishkar Pawar
[0.        0.        1.4732075 ... 0.        0.        0.       ]
Avishkar Pawar
[  0.         0.       159.023    ...   0.        45.243626   0.      ]
Avishkar Pawar
[  0.        0.        0.      ...   0.        0.      237.14462]
Avishkar Pawar
[0. 0. 0. ... 0. 0. 0.]
Avishkar Pawar
[  0.         0.         0.       ...   0.         0.       110.738655]
Avishkar Pawar
[  0.      0.    278.328 ...   0.      0.      0.   ]
Avishkar Pawar
[  0.       389.4469   101.198616 ...   0.         0.         0.      ]
Avishkar Pawar
[  0.        0.        0.      ...   0.      477.19467 256.21252]
Avishkar Pawar
[  0.        0.        0.      ...   0.      477.19467 256.21252]
Gal Gaddot
[  0.       0.

In [23]:
def predict(image):
    dist_list = [] #making distance list array
    R_embedding = deepface_model.predict(process_img(image))[0]
    for emb_db in embedding_list:
        euclidean_distance = findEuclideanDistance(normalize(R_embedding),normalize(emb_db))
        dist_list.append(euclidean_distance)
    min_dist = min(dist_list) #finding min distance
    print(min_dist)
    if min_dist<1.01: #condition for surity, this value is selected by us by oberving its value for differnet persons 
        min_dist_idx = dist_list.index(min_dist)
        name = name_list[min_dist_idx]
        return name
    else:
        name  = 'unkown'
        return name

## Realtime Face-Recognition

In [24]:
#intial cropping boundaries
l=0
r=0
t=0
b=0
open_door = False
detector = dlib.get_frontal_face_detector() #face detection using hog
cam = cv2.VideoCapture(0)
while True:
    ret, frame = cam.read()
    
    if not ret:
        print("no img error...")
        break
    
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) #converting image from BGR to RGB
        
    dect_box = detector(img_rgb) #identifies the face location and provides its location

            
    for box in dect_box:
        l=box.left()+15
        r=box.right()-15
        b=box.bottom()
        t=box.top()
        cv2.rectangle(frame, (box.left(), box.top()),(box.right(),box.bottom()), (0,255,0),1) # drawing rectangle around the face
    
    
    if (t<=0 or b<=0 or l<=0 or r<=0): #condition to avoid crashing when boundaries become negative
        cv2.imshow('video', frame)
        
    elif(not dect_box): #condition to avoid crashing when no face detected
        cv2.imshow('video', frame)


    else:
        crop_img = frame[t:b,l:r]
        Path('Test_face/').mkdir(parents=True, exist_ok = True)
        cv2.imwrite('Test_face/newface.jpg',crop_img)
        R_ImagePath='Test_face/newface.jpg'
        predicted_name = predict(R_ImagePath)
        cv2.putText(frame, predicted_name, (l,t-5),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0),2,cv2.LINE_AA) #writing name of the predicted person above the rectangle
        cv2.imshow('video', frame)
        
    p = cv2.waitKey(1) #closing the video feed
    if p%256 == 32:
        # Space pressed
        print("Analysing Person")
        person = predicted_name
        if person in name_list:
            open_door = True
            print("OPEN DOOR")
            break
        else:
            print("ACCESS DENIED")
        
    k = cv2.waitKey(1) #closing the video feed
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break

cam.release()
cv2.destroyAllWindows() 

1/1 [==============================] - 0s 44ms/step
0.8360094
1/1 [==============================] - 0s 41ms/step
0.7423272
1/1 [==============================] - 0s 39ms/step
0.76350677
1/1 [==============================] - 0s 43ms/step
1.0685346
1/1 [==============================] - 0s 39ms/step
0.80243295
1/1 [==============================] - 0s 39ms/step
1.0352652
1/1 [==============================] - 0s 41ms/step
0.77885485
1/1 [==============================] - 0s 42ms/step
0.7354994
1/1 [==============================] - 0s 44ms/step
0.9677424
1/1 [==============================] - 0s 44ms/step
0.9696328
1/1 [==============================] - 0s 39ms/step
0.93141824
1/1 [==============================] - 0s 43ms/step
0.71106935
1/1 [==============================] - 0s 41ms/step
0.681532
1/1 [==============================] - 0s 39ms/step
1.0078818
1/1 [==============================] - 0s 42ms/step
0.9526782
1/1 [==============================] - 0s 40ms/step
0.7267114
1/1 

In [ ]:
#Emergency brakes
cam.release()
cv2.destroyAllWindows()  